In [ ]:
from train_on_gei import *

In [ ]:
torch.cuda.set_device(0)

Train

In [ ]:
#set_seed(0)

In [ ]:
dataset = 'casiab-nm'
# ends of (tr,vl,ts) splits
splitset = '50,74,124'
# train with the first 74 persons
task,split = 'tr','tv'
# LB model
model = 'lb'
# MT model
#model = 'mt'
# Siamese model
#model = 's'

opt,lr,mom,wd,bs,epochs,sched = 'sgd',0.01,0.9,0.0005,128,240,'st-14-96'

In [ ]:
data,data_mean = get_data(dataset, splitset, bs, task, split)

In [ ]:
get_net = {'lb':LBNet,'mt':MTNet,'s':SiameseNet,'d':DebugNet}.get(model, None)
if get_net: net = get_net(data_mean=data_mean)
else: assert False, 'Not implemented for model {}.'.format(model)
model_dir = Path('output')/dataset
assert opt=='sgd', f'Unknown opt method {opt}'
opt_func = partial(SGDEx, momentum=mom)
learn = LearnerEx(data, net, opt_func=opt_func, metrics=accuracy,
                  true_wd=False, wd=wd, path=Path('..'), model_dir=model_dir)
learn.callback_fns[0] = partial(RecorderEx, add_time=learn.add_time)

In [ ]:
model_name = f'{dataset}_{model}_{opt}-{lr}-{mom}-{wd}_{sched}_bs{bs}_{split}'
assert sched.startswith('st')
iters = array([float(x)*10000 for x in sched.split('-')[1:]])
batches = len(data.train_dl) * epochs
assert np.all(iters < batches)
iters = np.append(iters, batches).astype(np.int)
phs = [TrainingPhase(x).schedule_hp('lr', lr*0.1**i) for i,x in enumerate(iters)]
learn.callback_fns += [
    partial(GeneralScheduler, phases=phs),
    partial(SaveModelCallback, every='epoch', name=model_name),
]

In [ ]:
learn.fit(epochs, 1)
learn.recorder.plot_losses()

Test

In [ ]:
data,_ = get_data(dataset, splitset, bs, 'ts', 'ts')

In [ ]:
learn = LearnerEx(data, net, metrics=accuracy, path=Path('..'), model_dir=model_dir)
learn.create_opt(lr, wd)
learn.callback_fns[0] = partial(RecorderEx, add_time=learn.add_time)
learn.callbacks += [learn.callback_fns[0](learn)]
learn.load('casiab-nm_lb_sgd-0.01-0.9-0.0005_st-14-96_bs128_tv_239', purge=False)
preds = learn.get_preds()

In [ ]:
xl = learn.data.valid_dl.x
# acc.shape is (probe,gallery)
acc = RecorderEx.calc_acc(learn.recorder.preds, xl.items1.inner_df, xl.items2.inner_df)
pacc = array([j[array(chain(range(i),range(i+1,acc.shape[1])))] for i,j in enumerate(acc)])
with np_print_options(formatter={'float':'{:1.7f}'.format}, threshold=sys.maxsize):
    print(pacc.mean())
    print(pacc.mean(1))
    print(acc)